### pipeline example to create volumes

In [ ]:
## new-volume.yaml
''' 
apiVersion: v1
kind: PersistentVolume
metadata:
 name: new-volume
spec:
 accessModes:
 - ReadWriteMany
 capacity:
   storage: 1Gi
 hostPath:
   path: /home/dkube/new-volume
 persistentVolumeReclaimPolicy: Retain
'''
## Run the below command on terminal
#kubectl create -f new-volume.yaml

In [ ]:
import kfp
import kfp.dsl as dsl

In [ ]:
@kfp.dsl.pipeline(
  name='volume-secrets',
  description='Simple-volume-secrets-pl'
)
def volumes_secret_pipeline():
    dvop = dsl.VolumeOp(name="create_pvc",resource_name="my-pvclaim",size="500Mi",modes=dsl.VOLUME_MODE_RWM,volume_name ='new-volume')
    step1 = dsl.ContainerOp(name="step1",image="library/bash:4.4.23",command=["sh", "-c"],arguments=["echo 1 | tee /mnt/file1"],pvolumes={"/mnt": dvop.volume})
    step2 = dsl.ContainerOp(name="step2",image="library/bash:4.4.23",command=["sh", "-c"],arguments=["echo 2 | tee /mnt2/file2"],pvolumes={"/mnt2": dvop.volume})
    step3 = dsl.ContainerOp(name="step3",image="library/bash:4.4.23",command=["sh", "-c"],arguments=["cat /mnt/file1 /mnt/file2"],pvolumes={"/mnt": dvop.volume.after(step1, step2)})


In [ ]:
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(volumes_secret_pipeline, arguments={})